# Convolutional Autoencoder on SSENSE Dataset

The purpose is to find similar products within the dataset based on the image similarities.

**Dataset:**
SSENSE dataset is consist of images of 224x224 pixels.<br>
Using Autoencoder, we are going to reconstruct images based on a trained model. In this case, these reconstructed images have less detailed-features, and they mostly represent eigen-features of their representative image. Since these images are dimensionally reduced, it would be much faster to compute the similarity/distance based on the reconstructed images instead of the original images.

* 1. A convolutional autoencoder has been designed to create a reconstructed images which are representative of image within the dataset.
* 2. Eigenfeature representative would be generated for all images in Test set.
* 3. One sample image is selected from Test set.
* 4. The eigenfeature output of the selected image would be compared with all eigenfeatures of all images, and their distance would be calculated.
* 5. Those products whose eigenfeatures are closest to the queried eigenfeature, are selected as the similar products.

### Libraries:
**Keras:** To build the model of convolutional autoencoder.<br>
**Matplotlib:** To plot the model performance and showing the images of eigenfeatures and actual images.<br>
**sklearn:** Using the KNN to find the nearest products.

In [1]:
import os, sys
import time
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

ModuleNotFoundError: No module named 'sklearn'

<hr>

### Loading Dataset
Numpy dataset is being loaded.

In [2]:
print("Loading dataset...")
t0 = time.time()

# loading numpy dataset
mypath = ".\\data"
dataset_file = os.path.dirname(mypath) + "\\dataset_data.npy"
total_data = np.load(dataset_file, allow_pickle=True, fix_imports=True)

print("Dataset loaded.")
print("{:.2f} seconds passed.".format(time.time() - t0))

Loading dataset...


NameError: name 'np' is not defined

<hr>

### Dataset Splitting
Loaded dataset is being splitted into a train set and a test set.<br>
The splitted datasets are being normalized.

In [25]:
t0 = time.time()
total_samples = len(total_data) #18256
print("Spliting train and test sets...")
# dataset splitting
x_train, x_test, _, _ = train_test_split(total_data, total_data, test_size=0.33, random_state=42)

print("Split completed.")

# reporting a summary of split
print("Train set: ", x_train.shape, " - {:.2f}%".format((x_train.shape[0] * 100)/total_samples))
print("Test set: ", x_test.shape, " - {:.2f}%".format((x_test.shape[0] * 100)/total_samples))

print ("Normalizing...")
# normalizing
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print ("Normalization completed.")
print("{:.2f} seconds passed.".format(time.time() - t0))
#x_train = np.reshape(x_train, (len(x_train), 224, 224, 1))  # adapt this if using `channels_first` image data format
#x_test = np.reshape(x_test, (len(x_test), 224, 224, 1))  # adapt this if using `channels_first` image data format

Spliting train and test sets...
Split completed.
Train set:  (12231, 224, 224, 1)  - 67.00%
Test set:  (6025, 224, 224, 1)  - 33.00%
Type converting...
Type conversion completed.
98.77 seconds passed.


<hr>

### Convolutional Autoencoder
Designing a convolutional autoencoder to build image representatives.<br>
Later the **SSENSE** dataset is loaded from Keras for training.<br>
The generated model would be saved at the end for future use.

In [2]:
input_img = Input(shape=(224, 224, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img) #224 x 224 x 16
x = MaxPooling2D((2, 2), padding='same')(x) #112 x 112 x 16

x = Conv2D(32, (3, 3), activation='relu', padding='same')(x) #112 x 112 x 32
x = MaxPooling2D((2, 2), padding='same')(x) #56 x 56 x 32
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x) #56 x 56 x 32
x = MaxPooling2D((2, 2), padding='same')(x) #28 x 28 x 32

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #28 x 28 x 64
x = MaxPooling2D((2, 2), padding='same')(x) #14 x 14 x 64
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #14 x 14 x 64
x = MaxPooling2D((2, 2), padding='same')(x) #7 x 7 x 64
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #7 x 7 x 64
encoded = MaxPooling2D((2, 2), padding='same')(x)  #4 x 4 x 64

# at this point the representation is (4, 4, 64) i.e. 1024-dimensional

x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded) #4 x 4 x 64
x = UpSampling2D((2, 2))(x) #8 x 8 x 64
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #8 x 8 x 64
x = UpSampling2D((2, 2))(x) #16 x 16 x 64
x = Conv2D(64, (3, 3), activation='relu')(x) #14 x 14 x 64
x = UpSampling2D((2, 2))(x) #28 x 28 x 64
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #28 x 28 x 64
x = UpSampling2D((2, 2))(x) #56 x 56 x 64

x = Conv2D(32, (3, 3), activation='relu', padding='same')(x) #56 x 56 x 32
x = UpSampling2D((2, 2))(x) #112 x 112 x 32
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x) #112 x 112 x 16
x = UpSampling2D((2, 2))(x) #224 x 224 x 16

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy') #adadelta

# printing model architecture
print(autoencoder.summary())

from keras.callbacks import History
history = History()

# training the dataset
autoencoder.fit(x_train, x_train,
               epochs=10,
               batch_size=128,
               shuffle=True,
               validation_data=(x_test, x_test), callbacks=[history])

# storing the model
autoencoder.save('C:\\Users\\kaveh.bakhtiyari\\JupyterProjects\\ImageSearch-ACE\\Models\\autoen_ssense.h5')
#del autoencoder








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 32)        9248      
_________________________________________

NameError: name 'x_train' is not defined

<hr>

### Performance
The performance of the model based on the **Loss** and **Val_Loss** is ploted.<br>
The model was tested based on the various types of neutwork, by extending the network (going deeper), but the performance did not change significantly.

In our tests, we had 200 epochs. There was a huge loss spike on 100 epochs, but the network recovered quickly.<br>
150 epochs recommended.

In [ ]:
print(history.history.keys())
for index, key in enumerate(history.history.keys()):
    print(key, history.history[key])
    plt.plot(history.history[key], label=key)

plt.ylabel(key)
plt.xlabel("Epoch")
plt.legend(loc='upper right')
plt.show()

<hr>

### Reconstructing & Sample Checking

All images within the test set would be reconstructed by autoencoder model.<br>
Then a sample of 10 images are ploted along with the originals and reconstructed-images to check how the model worked visually.

In [ ]:
# loading the saved model (if required)
from keras.models import load_model
autoencoder = load_model('.\\models\\autoen_ssense.h5')

# Generating the eigen-features (reconstructed images) for all images within test set
decoded_imgs = autoencoder.predict(x_test)

# Plotting n sample of images/eigen-features from test set
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    i = i + 1
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(224, 224))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(224, 224))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

<hr>

### Query (Nearest Neighbors)

At this stage, decoded images are reshaped to a linear array for calculating the distance.<br>
Then, a sample image from **Test Set** is selected, and its similarity distance is calculated against all the reconstructed images in the dataset.<br>
The closest items (*n_neighbors*) are extracted to be ploted as the most *n* similar products.

In [1]:
# Fit the NN algorithm to the encoded test set
x_t = np.zeros((len(decoded_imgs),50176))
for i in range(len(decoded_imgs)):
    x_t[i] = decoded_imgs[i].reshape(50176)

# fiting to find the nearest neighbors
nbrs = NearestNeighbors(n_neighbors=20).fit(x_t)

# selecting a sample product
sample_image = 200
query_image = np.zeros((1,50176))
query_image[0] = np.array(decoded_imgs[sample_image].reshape(50176))
#np.array(query_code)

#print (query_image.shape)
# finding the closest images to the encoded query image
distances, indices = nbrs.kneighbors(query_image)

print("Distances:", distances)
print("Indices:", indices)

NameError: name 'np' is not defined

<hr>

### Visualization

When the similar eigen-features are found, the actual product of those features are ploted.<br>
In this section, the followings are ploted:
* Original Image: The original queried image,
* Queried Image: The eigen-feature image of the queried image,
* Original Images of similar products to our query.

In [ ]:
# Plotting the original image
plt.figure(figsize=(5, 2))
ax = plt.subplot(1, 2, 1)
plt.subplots_adjust(bottom=0.1, top=1)
plt.imshow(x_test[sample_image].reshape(224, 224))
plt.xlabel("Original Image")

# Plotting the constructed image of the original image
ax = plt.subplot(1, 2, 2)
plt.imshow(decoded_imgs[sample_image].reshape(224, 224))
plt.gray()
plt.xlabel("Queried Image")
ax.get_yaxis().set_visible(False)
plt.show()

# Plotting the n origianl images which are found to be the nearest within the test-set
n = 40
plt.figure(figsize=(40, 8))
j = 0
for index, value in enumerate(indices[0]):
    # display original from test set
    j = j + 1
    ax = plt.subplot(int(n/10), 10, j)
    plt.subplots_adjust(bottom=0.1, top=1)
    plt.imshow(x_test[value].reshape(224, 224))
    plt.gray()
    plt.xlabel(value)
    ax.get_yaxis().set_visible(False)
    
    j = j + 1
    # display reconstructed
    ax = plt.subplot(int(n/10), 10, j)
    plt.imshow(decoded_imgs[value].reshape(224, 224))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()